In [ ]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yaml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=1.0.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - mlflow==2.8.0
    - mlflow-skinny==2.8.0
    - azureml-mlflow==1.51.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib
    - tensorflow

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    Model, Environment, ManagedOnlineEndpoint, ManagedOnlineDeployment
)
from azure.ai.ml.constants import AssetTypes
import datetime

# authenticate
credential = DefaultAzureCredential()

# Set up MLClient
SUBSCRIPTION = "7c47868f-0e47-4fed-8699-346d78083251"
RESOURCE_GROUP = "TECHIN515-lab"
WS_NAME = "TECHIN515"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# Register the model
model = ml_client.models.create_or_update(
    Model(
        path="wand_model.h5",
        name="wand-gesture-model",
        type=AssetTypes.CUSTOM_MODEL,
        description="Trained gesture model for wand detection"
    )
)

# Create or reuse an environment
env = Environment(
    name="wand-inference-env",
    description="Inference environment for wand model",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="dependencies/conda.yaml",
)
env = ml_client.environments.create_or_update(env)

# Create unique endpoint name using timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
endpoint_name = f"wand-endpoint-{timestamp}"

# Define the endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Wand gesture classification endpoint",
    auth_mode="key"
)
ml_client.begin_create_or_update(endpoint).result()


